# M2 - ML Competition

### Name: Ashish Nanda
### Roll No.: J041
### Batch - 2

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nmims-m2-batch-2/train.csv
/kaggle/input/nmims-m2-batch-2/test.csv


In [2]:
train = pd.read_csv('../input/nmims-m2-batch-2/train.csv')

test = pd.read_csv('../input/nmims-m2-batch-2/test.csv')

In [3]:
train.head()

,id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,137901,1994-09,HOUGANG,4 ROOM,443,HOUGANG AVE 8,07 TO 09,103.0,MODEL A,1990,193000
1,47497,1999-06,ANG MO KIO,3 ROOM,461,ANG MO KIO AVE 10,07 TO 09,68.0,NEW GENERATION,1980,150000
2,250681,1995-06,BEDOK,5 ROOM,67,BEDOK STH AVE 3,07 TO 09,123.0,STANDARD,1977,340000
3,227426,1996-03,BEDOK,3 ROOM,530,BEDOK NTH ST 3,04 TO 06,68.0,NEW GENERATION,1979,153000
4,277594,1994-10,BISHAN,3 ROOM,306,SHUNFU RD,04 TO 06,71.0,NEW GENERATION,1985,116000


In [4]:
test.head()

,id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date
0,5,1997-12,TOA PAYOH,3 ROOM,96,LOR 3 TOA PAYOH,13 TO 15,68.0,IMPROVED,1970
1,6,1997-02,JURONG EAST,5 ROOM,53,TEBAN GDNS RD,01 TO 03,121.0,IMPROVED,1986
2,15,1999-04,BEDOK,3 ROOM,71,BEDOK STH RD,07 TO 09,67.0,NEW GENERATION,1978
3,20,1995-06,YISHUN,3 ROOM,133,YISHUN ST 11,04 TO 06,68.0,NEW GENERATION,1984
4,21,1992-11,JURONG WEST,3 ROOM,211,BOON LAY PL,10 TO 12,65.0,IMPROVED,1976


In [5]:
train.shape, test.shape

((201037, 11), (86159, 10))

#### Merging the train and test datasets for pre-processing

In [6]:
test['resale_price'] = -1 

test_ID = test['id']

df = pd.concat([train, test], axis=0)

df.drop('id', axis=1, inplace=True)

In [7]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,1994-09,HOUGANG,4 ROOM,443,HOUGANG AVE 8,07 TO 09,103.0,MODEL A,1990,193000
1,1999-06,ANG MO KIO,3 ROOM,461,ANG MO KIO AVE 10,07 TO 09,68.0,NEW GENERATION,1980,150000
2,1995-06,BEDOK,5 ROOM,67,BEDOK STH AVE 3,07 TO 09,123.0,STANDARD,1977,340000
3,1996-03,BEDOK,3 ROOM,530,BEDOK NTH ST 3,04 TO 06,68.0,NEW GENERATION,1979,153000
4,1994-10,BISHAN,3 ROOM,306,SHUNFU RD,04 TO 06,71.0,NEW GENERATION,1985,116000


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287196 entries, 0 to 86158
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                287196 non-null  object 
 1   town                 287196 non-null  object 
 2   flat_type            287196 non-null  object 
 3   block                287196 non-null  object 
 4   street_name          287196 non-null  object 
 5   storey_range         287196 non-null  object 
 6   floor_area_sqm       287196 non-null  float64
 7   flat_model           287196 non-null  object 
 8   lease_commence_date  287196 non-null  int64  
 9   resale_price         287196 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 24.1+ MB


In [9]:
df.describe()

,floor_area_sqm,lease_commence_date,resale_price
count,287196.000000,287196.000000,287196.000000
mean,93.351778,1983.206897,153631.270874
std,27.361872,6.085632,146958.792542
min,28.000000,1967.000000,-1.000000
25%,68.000000,1979.000000,-1.000000
50%,91.000000,1984.000000,137000.000000
75%,113.000000,1987.000000,250000.000000
max,307.000000,1997.000000,883000.000000


In [10]:
df.var()

floor_area_sqm         7.486720e+02
lease_commence_date    3.703492e+01
resale_price           2.159689e+10
dtype: float64

#### The target variable has high variance

In [11]:
df.isna().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
dtype: int64

#### There are no missing values present in the dataset

## Feature Engineering

In [12]:
df['flat_type'].value_counts()

3 ROOM              113102
4 ROOM               98521
5 ROOM               48206
EXECUTIVE            22121
2 ROOM                4140
1 ROOM                 827
MULTI GENERATION       279
Name: flat_type, dtype: int64

#### Creating a feature that further groups the types of flat into 3 categories

In [13]:
def Grouping_Flat(row):
    if row == '1 ROOM' or row == '2 ROOM' or row == '3 ROOM':
        row = 'Type 1'
    
    elif row == '4 ROOM' or row == '5 ROOM':
        row = 'Type 2'
    
    else:
        row = 'Type 3'
        
    return row

df['Flat_Type'] = df['flat_type'].apply(Grouping_Flat)

#### Creating a feature to compute the ratio of floor area to number of rooms, by extracting the number of rooms from 'flat_type' column

In [14]:
def Extract_Rooms(row):
    if row == 'MULTI GENERATION':
        row = 5
    
    elif row == 'EXECUTIVE':
        row = 1
    
    else:
        row = int(row[0])
        
    return row

df['Room_Count'] = df['flat_type'].apply(Extract_Rooms)

df['Ratio'] = df['floor_area_sqm'] / df['Room_Count']

#### Creating a feature that takes the average of upper and lower storey

In [15]:
def Split_Storey(row):
    left_term = int(row.split(sep=' TO ')[0])
    right_term = int(row.split(sep=' TO ')[1])
    row = int(np.mean([left_term, right_term]))
        
    return row

df['Mean_Storey'] = df['storey_range'].apply(Split_Storey) 

#### Creating a feature to split the 'month' column to month and year

In [16]:
def Split_Date_Month(row):
    row = row.split(sep='-')[1]
    row = int(row)
        
    return row

df['Month'] = df['month'].apply(Split_Date_Month)

In [17]:
def Split_Date_Year(row):
    row = row.split(sep='-')[0]
    row = int(row)
        
    return row

df['Year'] = df['month'].apply(Split_Date_Year)

#### Creating a feature that calculates the difference between lease commencement and end date  

In [18]:
df['Duration'] = df['Year'] - df['lease_commence_date']

#### Unfortunately, most of the features created didn't have a positive impact on model's performance 

In [19]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,Flat_Type,Room_Count,Ratio,Mean_Storey,Month,Year,Duration
0,1994-09,HOUGANG,4 ROOM,443,HOUGANG AVE 8,07 TO 09,103.0,MODEL A,1990,193000,Type 2,4,25.750000,8,9,1994,4
1,1999-06,ANG MO KIO,3 ROOM,461,ANG MO KIO AVE 10,07 TO 09,68.0,NEW GENERATION,1980,150000,Type 1,3,22.666667,8,6,1999,19
2,1995-06,BEDOK,5 ROOM,67,BEDOK STH AVE 3,07 TO 09,123.0,STANDARD,1977,340000,Type 2,5,24.600000,8,6,1995,18
3,1996-03,BEDOK,3 ROOM,530,BEDOK NTH ST 3,04 TO 06,68.0,NEW GENERATION,1979,153000,Type 1,3,22.666667,5,3,1996,17
4,1994-10,BISHAN,3 ROOM,306,SHUNFU RD,04 TO 06,71.0,NEW GENERATION,1985,116000,Type 1,3,23.666667,5,10,1994,9


#### Encoding categorical columns

In [20]:
df_categorical = df.select_dtypes('object').columns
df_categorical = list(df_categorical)

encoder = LabelEncoder()

for col in df_categorical:
    df[col] = encoder.fit_transform(df[[col]])

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 287196 entries, 0 to 86158
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                287196 non-null  int64  
 1   town                 287196 non-null  int64  
 2   flat_type            287196 non-null  int64  
 3   block                287196 non-null  int64  
 4   street_name          287196 non-null  int64  
 5   storey_range         287196 non-null  int64  
 6   floor_area_sqm       287196 non-null  float64
 7   flat_model           287196 non-null  int64  
 8   lease_commence_date  287196 non-null  int64  
 9   resale_price         287196 non-null  int64  
 10  Flat_Type            287196 non-null  int64  
 11  Room_Count           287196 non-null  int64  
 12  Ratio                287196 non-null  float64
 13  Mean_Storey          287196 non-null  int64  
 14  Month                287196 non-null  int64  
 15  Year              

#### Splitting the concatenated data back to train and test 

In [22]:
test = df[df['resale_price'] == -1]
test = test.drop('resale_price', axis=1)

train = df[df['resale_price'] != -1]

In [23]:
train['resale_price'].value_counts()

47000     1878
160000    1831
150000    1830
180000    1782
140000    1695
          ... 
195750       1
263998       1
141888       1
153300       1
194688       1
Name: resale_price, Length: 3297, dtype: int64

In [24]:
list(df.columns)

['month',
 'town',
 'flat_type',
 'block',
 'street_name',
 'storey_range',
 'floor_area_sqm',
 'flat_model',
 'lease_commence_date',
 'resale_price',
 'Flat_Type',
 'Room_Count',
 'Ratio',
 'Mean_Storey',
 'Month',
 'Year',
 'Duration']

In [25]:
features = ['month',
 'town',
 'flat_type',
 'block',
 'street_name',
 'storey_range',
 'floor_area_sqm',
 'flat_model',
 'lease_commence_date',
 'Duration']

In [26]:
test = test[features]

In [27]:
num_bins = int(np.floor(1 + np.log2(len(train))))

train['bins'] = pd.cut(train['resale_price'], bins=num_bins, labels=False)

In [28]:
# # log normalisation of target

# train['resale_price'] = np.log(train['resale_price'])


#### Log normalisation of target didn't improve the model. :(

In [29]:
X = train[features]

y = train['resale_price']

In [30]:
X.shape, y.shape

((201037, 10), (201037,))

In [31]:
# def score(y_true, y_pred):
#     score = mean_absolute_error(y_true, y_pred)
#     return score

In [32]:
# # Model Hyperparameter tuning

# skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=24)

# parameters = {'depth' : np.arange(6, 16, 2) ,
#               'learning_rate' : [0.01, 0.05, 0.1, 0.5],
#               'iterations' : [50, 100, 150, 200]}

# model = CatBoostRegressor(task_type='GPU', verbose=0) 

# grid_search = GridSearchCV(estimator=model, param_grid=parameters, cv=skf.split(X, y=train['bins'].values), n_jobs=-1, scoring=make_scorer(score, greater_is_better=True), verbose=10)

# grid_search.fit(X, y)

In [33]:
# print(f'Best estimator: {grid_search.best_estimator_}')
# print(f'Best score: {grid_search.best_score_}')
# print(f'Best parameters: {grid_search.best_params_}')

## Cross Validation and Model Fitting

In [34]:
# best_params = grid_search.best_params_

pred = np.zeros((test.shape[0]))

skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=24)

scores = []

for train_index, test_index in tqdm(skf.split(X, y=train['bins'].values)):
    
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]

    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]  


#   model = XGBRegressor(max_depth=8, task_type='GPU')
    model = CatBoostRegressor(max_depth=10, verbose=0)

    model.fit(X_train_fold, y_train_fold)

    y_pred = model.predict(X_test_fold)
    
    
#     y_test_fold = [(np.exp(x)) for x in [i for i in y_test_fold]]

#     y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

    
    pred += model.predict(test) / skf.n_splits 

    score = mean_absolute_error(y_test_fold, y_pred)
        
    scores.append(score)

7it [04:11, 35.95s/it]


In [35]:
print(f'Average K-Folds MAE: {np.mean(scores)}')

Average K-Folds MAE: 11204.756612649138


In [36]:
# Submission File for K-Folds

submission = pd.DataFrame({'id':test_ID, 'resale_price':pred})
submission.to_csv('Submission7.csv', index=False)

In [37]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=22, shuffle=True)

In [38]:
# model = CatBoostRegressor(verbose=0)

# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

In [39]:
# # With log normalisation of target

# model = CatBoostRegressor(verbose=0)

# model.fit(X_train, y_train)

# y_pred_log = model.predict(X_test)

# y_test = [(np.exp(x)) for x in [i for i in y_test]]

# y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

In [40]:
# score = mean_absolute_error(y_test, y_pred)
# print(f'MAE: {score}') 

In [41]:
# # Submission File

# model.fit(X, y)

# y_pred = model.predict(test)

# submission = pd.DataFrame({'id':test_ID, 'resale_price':y_pred})
# submission.to_csv('Submission1.csv', index=False)

In [42]:
# # Submission File with log normalisation of target

# model.fit(X, y)

# y_pred_log = model.predict(test)

# y_pred = [(np.exp(x)) for x in [i for i in y_pred_log]]

# submission = pd.DataFrame({'id':test_ID, 'resale_price':y_pred})
# submission.to_csv('Submission2.csv', index=False)